In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import proplot as pplt
from scipy.fftpack import * 
import glob

# 进行每小时气温数据的处理

获取每小时气温数据，并且计算3hr 6hr的取样频率下的每日平均、最大、最小气温

需要手动进行取样时间的设置

In [2]:
def get_hourly_t2m(modeltype = "VR"):
    """
    获取逐小时的T2M，并且得到3小时间隔的mean min max结果
    """
    path_in      = "/raid52/yycheng/MPAS/"+modeltype+"_postprocess/"+modeltype+"_merge/t2m_hourly/"
    files_in     = glob.glob(path_in + "merge_t2m_????_"+modeltype+".nc")
    da_hourlyt2m = xr.open_mfdataset(files_in[:],engine = 'netcdf4')['t2m'].load()

    # time_ind_3hr = ( (da_hourlyt2m.Time.dt.hour.isin([0,3,6,9,12,15,18,21])) & (da_hourlyt2m.Time.dt.month.isin([4,5,6,7,8])) )
    time_ind_3hr = ( (da_hourlyt2m.Time.dt.hour.isin([0,6,12,18])) & (da_hourlyt2m.Time.dt.month.isin([4,5,6,7,8])) )
    temp_3hr     = da_hourlyt2m.isel(Time = time_ind_3hr)

    temp_3hr_daily = {}
    temp_3hr_daily['mean'] = temp_3hr.groupby(temp_3hr.Time.dt.date).mean()
    temp_3hr_daily['max']  = temp_3hr.groupby(temp_3hr.Time.dt.date).max()
    temp_3hr_daily['min']  = temp_3hr.groupby(temp_3hr.Time.dt.date).min()
    really_time = pd.to_datetime(temp_3hr_daily['mean'].date.values)
    for i in temp_3hr_daily:
        temp_3hr_daily[i] = temp_3hr_daily[i].rename({"date":"Time"}).assign_coords({"Time":really_time})

    return temp_3hr_daily

In [3]:
test_dict = get_hourly_t2m(modeltype='VR')

In [ ]:
modeltype = "VR"
path_out = "/raid52/yycheng/MPAS/REFERENCE/TEMP_DATA/mask_t2m/ordata_6hr/"
test_dict['mean'].to_netcdf(path_out + "mean_t2m_98-17_"+modeltype+".nc")
test_dict['max'].to_netcdf(path_out + "max_t2m_98-17_"+modeltype+".nc")
test_dict['min'].to_netcdf(path_out + "min_t2m_98-17_"+modeltype+".nc")